# Introduction

The purpose of this notebook is to present the basics of Spark: why we use it, and what kind of operations we can perform using it.

In this notebook we cover the following topics:
- SparkContext object
- RDDs
- Operations on RDDs:
    - transformations
    - actions
- DataFrames

## SparkContext

First of all, we need to create a driver Spark application which communicates the user commands to the Spark Workers. We do so with the help of the `SparkContext` object, which coordinates with the cluster manager about the resources required for execution and the actual tasks that need to be executed. The resources required can be defined within the `SparkConf` configuration object, which we pass as a parameter to the `SparkContext` object upon creation.

Once we create a `SparkContext (sc)` object, we use it for orchestrating the allocated resources. The process is illustrated on the image below. Each of the components on the image run in a different process, enabling parallel execution of the worker nodes. This setup is replicated for all applications submitted to the cluster, and due to the process isolation, they cannot communicate directly (exchange data) among each other without persisting it to the disk.

For more information on how Spark applications are deployed, please refer to the following resources:
- [Submitting Spark Applications](https://spark.apache.org/docs/latest/submitting-applications.html)
- [Cluster Mode Overview](https://spark.apache.org/docs/latest/submitting-applications.html)

![SparkContext overview](data/cluster_overview.png)

Image source: https://spark.apache.org/docs/latest/cluster-overview.html





In [5]:
import sys
from random import random
from pyspark import SparkContext, SparkConf

# Create a SparkConf configuration object which sets the Spark Application name
conf = SparkConf().setAppName("Spark Intro")

# Create the SparkContext object.
# In this case we are using `.getOrCreate` method to be able to rerun the same cell multiple times
sc = SparkContext.getOrCreate(conf=conf) # Alternatively, use SparkContext(conf=conf)

## Resilient Distributed Datasets - RDDs

Resilient Distributed Dataset (RDD) is the elementary data structure used in Spark. RDDs are immutable and fault-tolerant, meaning that once loaded, the data cannot be changed, and because of that the system is able to recalculate results from failing nodes. RDDs also enable operations on the enclosed data to be executed in parallel on multiple nodes.

There are 3 main ways of creating RDDs:
1. From an existing collection - parallelize the existing collection from the target programming language, such as an array.
2. Transforming an existing RDD - applying transformations on an RDD yields a new RDD.
3. Loading an external dataset - ability to load data from an existing source (e.g. from a file system)

There are 2 types of operations that can be applied to RDDs:
1. Transformations - a lazy operation which yields a new RDD
2. Actions - return a value to the driver program, i.e. execute all the predefined lazy operations of the RDD

We are going to explore all of these options in the examples below.


In [6]:
# Finally, stop the application.
sc.stop()